In [ ]:
!rm -rf sample_data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torchaudio
!pip install speechbrain

In [ ]:
#Import necessary libraries for loading data
import torch
import torchaudio
import numpy

In [ ]:
!rm -rf LibriMix/

In [ ]:
!mkdir LibriMix

DataLoader

In [ ]:
from torch.utils.data import Dataset
import os
from typing import Tuple, Union
from torch import Tensor

class MixDataset(Dataset):
    def __init__(self, wav_dir):
        self.wav_dir = wav_dir
        self.wav_files = os.listdir(wav_dir)

    def __len__(self):
        return len(self.wav_files)

    def __getitem__(self, idx)-> Tuple[Tensor, int, str, int, int, int]:
        file_audio = os.path.join(self.wav_dir, self.wav_files[idx])
        waveform, sample_rate = torchaudio.load(file_audio) 
        file_id = self.wav_files[idx]
        file_id = file_id.split("_")[0]
        speaker_id, chapter_id, utterance_id = file_id.split("-")
        utterance = ""
        for data in data_loader:
            if int(speaker_id) == int(data[3]) and int(chapter_id) == int(data[4]) and int(utterance_id) == int(data[5]):
               utterance = data[2]
               break
         
        return (waveform, sample_rate, utterance, int(speaker_id), int(chapter_id), int(utterance_id))

Mixdataset = MixDataset("/content/LibriMix/")
mixdata_loader = torch.utils.data.DataLoader(Mixdataset, batch_size=1)
print(len(Mixdataset))

3


In [ ]:
import torch
subset = torch.utils.data.Subset(Mixdataset, range(0, len(Mixdataset), 2))
print(len(subset))

2


In [ ]:
for data in mixdata_loader:
    print(str(data[2]))

In [ ]:
dataset = torchaudio.datasets.LIBRISPEECH(".", 'test-clean', download=True)
print(len(dataset))
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1)

ASR

In [ ]:
from speechbrain.pretrained import EncoderDecoderASR
from speechbrain.pretrained import SepformerSeparation
asr_model = EncoderDecoderASR.from_hparams(source="speechbrain/asr-crdnn-transformerlm-librispeech", savedir="./pretrained_ASR")
separator = SepformerSeparation.from_hparams(source="speechbrain/sepformer-wsj02mix", savedir="./pretrained_sepformer")

In [ ]:
import torchaudio
s1, fs = torchaudio.load('/content/LibriMix/777-126732-0057_84-121550-0019.wav')

rel_length = torch.tensor([1.0])
predicted_words, predicted_tokens = asr_model.transcribe_batch(s1, rel_length)
print(predicted_words[0])


resampler = torchaudio.transforms.Resample(8000, 8000)
s1 = resampler(s1)
est_sources = separator.separate_batch(s1)
est_sources = est_sources[0]
s1 = est_sources[:, 1]
s1 = s1 [None, :]
rel_length = torch.tensor([1.0])
predicted_words, predicted_tokens = asr_model.transcribe_batch(s1, rel_length)
print(predicted_words[0])

AND I THINK I'VE BEEN A PRINCIPAL IN MY DRINKING SYMPATHY I COULDN'T HAVE BEEN TO ME IF I HADN'T BEEN CRUSHED I'VE BEEN CRUSHED I'VE BEEN CRUSHED I'VE BEEN CRUSHED I'VE BEEN CRUSHED I'VE BEEN CRUSHED I'VE BEEN CRUSHED I'VE BEEN CRUSHED I'VE BEEN A CLUTCHING
AND I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I HAVEN'T I


In [ ]:
import IPython.display as ipd
ipd.Audio(est_sources[:, 1], rate=8000)

In [ ]:
!pwd

In [ ]:
import IPython.display as ipd
resampler = torchaudio.transforms.Resample(16000, 8000) 
resampler2 = torchaudio.transforms.Resample(8000, 16000) 
#s1 = resampler(s1)
#fs= 8000  
#est_sources = separator.separate_batch(s1)
#est_sources = est_sources[0]

In [ ]:
subset = torch.utils.data.Subset(dataset, range(0, len(dataset), 2)[:500])
data_loader2 = torch.utils.data.DataLoader(subset, batch_size=1)
print(len(subset))

500


In [ ]:
import torchaudio
s1, fs = torchaudio.load('/content/LibriMix/2902-9006-0004_3752-4944-0036.wav')
print(s1.shape)
# we resample because we will use a model trained on 8KHz data. 
resampler = torchaudio.transforms.Resample(fs, 8000) 
resampler2 = torchaudio.transforms.Resample(8000, 16000)

torch.Size([1, 54800])


In [ ]:
!pip install jiwer

In [ ]:
import jiwer
import numpy as np

avreageWER = 0
count = 0
for data in data_loader2:
   
  s1, fs = torchaudio.load('/content/LibriMix/2902-9006-0004_3752-4944-0036.wav')
  s2 = data[0][0, : , :]

  if s1.shape[1] < s2.shape[1]:
       s1 = s1[:s2.shape[1]]
       s1 = np.pad(s1, ((0, 0), (0, s2.shape[1] - s1.shape[1])), mode='constant',  constant_values=0)
  if s1.shape[1] > s2.shape[1]:
       s1 = s1[0,:s2.shape[1]]

  #s1 = torch.Tensor(s1)
  #s1 = resampler(s1)
  #s2 = resampler(s2)
  #mix = s1 + s2
  
  est_sources = separator.separate_batch(data[0][0, : , :])
  est_sources = est_sources[0] 
  #print(est_sources [:, 1].shape)
  mix = est_sources [None, :, 1]
  
  batch= asr_model.audio_normalizer(data[0], 16000)
  rel_length = torch.tensor([1.0])
  predicted_words, predicted_tokens = asr_model.transcribe_batch(mix, rel_length)
  wer = jiwer.wer(str(data[2]), predicted_words[0])

  count = count + 1
  print(count)

  avreageWER = avreageWER + wer

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [ ]:
import IPython.display as ipd
ipd.Audio(mix[0], rate=16000)

In [ ]:
print(avreageWER/len(subset))

1.0000252213650684
